In [ ]:
!pip install ultralytics
import cv2
import shutil
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
from os.path import isfile


In [ ]:
def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    fps = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1

        if frame_number == 1:
            start_time = cv2.getTickCount()
        
    end_time = cv2.getTickCount()
    time = (end_time - start_time) / cv2.getTickFrequency()
    fps = frame_number / (time*10)

    cap.release()
    return frame_array, fps

In [ ]:
def objectdetect(source_images):
    output_folder = 'temp_images'
    os.makedirs(output_folder, exist_ok=True)

    # Save the images from the array to the folder
    for i, image in enumerate(source_images):
        image_filename = os.path.join(output_folder, f'image_{i}.jpg')
        cv2.imwrite(image_filename, image)

    model = YOLO('yolov8n.pt')
    model = YOLO('yolov8n-pose.pt')# Pretrained YOLOv8n model
    model(source='temp_images', save=True, project='Project')
    #!yolo task=detect mode=predict model=yolov8n.pt source="temp_images"
    output_images = []

    project_directory = 'project'
  
    result_files = os.listdir(os.path.join('/kaggle/working/Project/predict'))

    sorted_result_files = sorted(result_files, key=lambda x: int(os.path.splitext(x)[0].split('_')[1]))
    
    # Load the sorted image files into an array
    output_images = [cv2.imread(os.path.join('/kaggle/working/Project/predict', filename)) for filename in sorted_result_files]
    shutil.rmtree('/kaggle/working/Project')
    return output_images

In [ ]:
def frame2video(dehazed_array, pathOut, fps=40):
    size = (dehazed_array[0].shape[1], dehazed_array[0].shape[0])
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(dehazed_array)):
        out.write(dehazed_array[i])
    out.release()

In [ ]:
import os
def main(input_video, output_video):
    frame_array,fps = video2framesarray(input_video)[0],video2framesarray(input_video)[1]
    dehazed_array = objectdetect(frame_array)
#     extract_features(frame_array)
    frame2video(dehazed_array, output_video,fps)
    
if __name__ == '__main__':
    input_video = '/kaggle/input/footballtest/football_test.mp4'
    output_video = 'output_video.mp4' 
    main(input_video, output_video)

In [ ]:
import cv2
import os
import shutil
from ultralytics import YOLO
def objectdetectimg(input_dir):
    
    model = YOLO('yolov8n.pt')
    model = YOLO('yolov8n-pose.pt')# Pretrained YOLOv8n model
    model(source= input_dir, save=True, project='Project' )

    #!yolo task=detect mode=predict model=yolov8n.pt source="temp_images"
    output_images = []

    project_directory = 'project'
  
    result_files = os.listdir(os.path.join('/kaggle/working/Project/predict'))

    sorted_result_files = sorted(result_files, key=lambda x: int(os.path.splitext(x)[0].split('_')[1]))
    
    # Load the sorted image files into an array
    output_images = [cv2.imread(os.path.join('/kaggle/working/Project/predict', filename)) for filename in result_files]
    shutil.rmtree('/kaggle/working/Project')
    return output_images

In [ ]:
def main(input_dir):
    dehazed_array = objectdetectimg(input_dir)
    
if __name__ == '__main__':
    input_dir = '/kaggle/input/soccer-dataset/Soccer Dataset/Test/Nonfoul'
    main(input_dir)

In [ ]:
!zip -r testnonfoul.zip /kaggle/working/Project/predict

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink
FileLink(r'testnonfoul.zip')